In [ ]:
#@title Install unsloth (model provider), accelerate and bitsandbytes (for quantization)

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" trl peft accelerate bitsandbytes

In [ ]:
#@title Loading the Pre-trained model
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#@title Applying LoRA adapters - Fine tuning the model
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
#@title Pre-processing the dataset
from google.colab import drive
import csv
drive.mount('/content/drive/')
import pandas as pd
from nltk.corpus import stopwords

data=pd.read_csv("/content/drive/My Drive/traindata.csv")
print(data.columns)
def clean_text(text):
  # Convert text to lowercase
  text = text.lower()
  # Remove punctuation
  text = "".join([char for char in text if char.isalnum() or char.isspace()])
  return text

# Preprocess Hook and Build up text
data["Hook"] = data["Hook"].apply(clean_text)
data["Build Up"] = data["Build Up"].apply(clean_text)
data['Body']= data["Body"].apply(clean_text)


data.to_csv("/content/drive/My Drive/traindata.csv",index=False)

In [ ]:
#@title Loading the dataset from the hugging_face datacard and prompting the model with the format required
alpaca_prompt = """Below is a Topic for a social media post. Generate the script like given in the Hook,Build Up,Body.

### Topic:
{}

### Script:
{}

"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    iny = examples["Topic"]
    instructions = examples["Hook"]
    inputs       = examples["Build Up"]

    outputs      = examples["Body"]
    texts = []
    for i,instruction, input, output in zip(iny,instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(i,instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("Surya1523/data_man", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)
print(dataset['Hook'])

In [ ]:
#@title Fixing the training hyperparameters
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
#@title Training the model
import xformers

print(xformers.__version__)
trainer_stats = trainer.train()

In [ ]:
trainer_stats.metrics

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
#@title Testing the fine-tuned model
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Education",
        ""
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

In [ ]:
!pip install gradio


In [ ]:
#@title Deploying with the Gradio UI
import gradio as gr
def generate_script(topic, hook, build_up, body):
  """Generates a social media post script based on the provided elements."""
  prompt = alpaca_prompt.format(topic,"", body)
  inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)  # Adjust max_new_tokens as needed
  return tokenizer.batch_decode(outputs)[0]

interface = gr.Interface(
  fn=generate_script,
  inputs=[
      gr.Textbox(label="Topic"),

  ],
  outputs="text",
  title="AI Social Media Script Generator",
  description="Generate creative social media post scripts based on your input."
)

interface.launch()